y = 3 * x + 2 (linear regressin)

In [14]:
import random


xs = [random.uniform(-50, 50) for _ in range(100)]
ys = [3 * x + 2 + random.uniform(-1, 1) for x in xs]
print("Generated 100 points")

Generated 100 points


In [15]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    def __radd__(self, other):
        return self + other


    def __neg__(self): return self * -1
    def __sub__(self, other): return self + (-other)
    def __truediv__(self, other): return self * other**-1
    def __pow__(self, power): 
        assert isinstance(power, (int, float)), "Only supports int/float powers"
        out = Value(self.data**power, (self,), f'**{power}')
        def _backward():
            self.grad += power * self.data**(power - 1) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [16]:
# Initialize weight and bias
w = Value(random.uniform(-1, 1))
b = Value(random.uniform(-1, 1))


In [17]:
for epoch in range(100):
    # Forward pass
    ypred = [(w * x + b) for x in xs]
    loss = sum((yp - ygt)**2 for yp, ygt in zip(ypred, ys)) / len(xs)

    # Backward pass
    for param in [w, b]:
        param.grad = 0
    loss.backward()

    # Gradient descent
    lr = 0.01
    for param in [w, b]:
        param.data -= lr * param.grad

    if epoch % 10 == 0 :
        print(f"Epoch {epoch} | Loss: {loss.data:.4f} | w: {w.data:.4f}, b: {b.data:.4f}")


Epoch 0 | Loss: 8318.6684 | w: 49.6436, b: -0.2967


Epoch 10 | Loss: 716997456918721694711939072.0000 | w: 13699520090640.9941, b: -40019084537.5564
Epoch 20 | Loss: 61835976311937695779082586173477076968073341173760.0000 | w: 4023158632114498490597376.0000, b: -11752464636883199066112.0000
Epoch 30 | Loss: 5332917055079550267610744515986740126922608740204039488276681918081138688.0000 | w: 1181487035463270853473499007978307584.0000, b: -3451363933894930558874222579417088.0000
Epoch 40 | Loss: 459926502541011967286951489637881311461232137448510335084279028486816449445828252019963142864896.0000 | w: 346969071471617403947818599642187792269126402048.0000, b: -1013567228001445073556078306456735502414381056.0000
Epoch 50 | Loss: 39665418673280397866977140629549442850357182464732565640632938513326051204779582850334128997396737107654742881559642112.0000 | w: 101894928123922481802423172468803702635139050947609389694976.0000, b: -297655809516206827332263531221361460237357253373661806592.0000
Epoch 60 | Loss: 34208627461869673946907228254807329728

In [8]:
print(f"Final Model: y = {w.data:.2f} * x + {b.data:.2f}")


Final Model: y = 3.00 * x + 1.70
